In [1]:
import os
import numpy as np
import pandas as pd
import re

from utils.logger import LOGPATH

In [16]:
fpath = LOGPATH.joinpath('summary-fbd.csv')
tori = pd.read_csv(fpath, engine='python')
tori

,seed,data,model,conv,epoch_best,epoch_learn,time_learn,mem_ram_train,mem_cuda_train,f1macro_train,...,f1micro_test,time_eval,mem_ram_eval,mem_cuda_eval,f1macro_high,f1micro_high,f1macro_low,f1micro_low,num_high,num_low
0,10,cora,MLP,Identity,91,500,1.9729,1.075,0.039,100.000,...,76.895,0.0002,1.075,0.039,70.231,76.000,74.705,77.098,100,441
1,11,cora,MLP,Identity,54,500,1.1905,1.156,0.039,99.870,...,76.710,0.0002,1.156,0.039,69.175,72.500,75.616,77.910,120,421
2,12,cora,MLP,Identity,52,500,1.1884,1.156,0.039,99.365,...,77.079,0.0002,1.156,0.039,81.036,82.456,74.125,75.644,114,427
3,10,cora,DecoupledFixed,AdjConv-impulse,79,500,3.6606,1.098,0.055,59.090,...,72.458,0.0010,1.098,0.055,52.007,64.000,57.652,74.376,100,441
4,11,cora,DecoupledFixed,AdjConv-impulse,30,500,2.8227,1.179,0.055,45.589,...,64.140,0.0008,1.179,0.055,46.603,67.227,43.502,63.270,119,422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,11,actor,DecoupledVar,BernConv,62,500,18.1604,1.217,0.545,31.441,...,35.855,0.0072,1.217,0.545,28.019,34.797,26.564,36.111,296,1224
311,10,pubmed,MLP,Identity,124,500,2.0773,1.080,0.099,99.986,...,87.595,0.0002,1.080,0.099,84.064,85.025,88.116,88.237,788,3154
312,11,pubmed,MLP,Identity,148,500,1.3241,1.233,0.099,99.993,...,87.418,0.0002,1.233,0.099,84.757,85.195,87.945,87.957,770,3172
313,12,actor,DecoupledVar,BernConv,49,500,16.1494,1.219,0.545,34.169,...,35.066,0.0072,1.219,0.545,22.824,31.210,25.494,36.070,314,1206


In [17]:
dlst = ["cora", "citeseer", "chameleon_filtered", "squirrel_filtered", "pubmed", "actor"]
id_cols = ["seed", 'data', 'model', 'conv',]
m_cols = ['deg_high', 'deg_low', 's_f1i_high', 's_f1i_low', 's_f1i_test',]
t = tori[tori['data'].isin(dlst)][id_cols+m_cols].sort_values(by=['data', 'model', 'conv', 'seed'])
t.sort_index(ascending=False).groupby(['data', 'model', 'conv']).filter(lambda x: len(x) > 3)

,seed,data,model,conv,num_high,num_low,f1micro_high,f1micro_low,f1micro_test


In [18]:
conv_repr = {
    'MLP': {
        'Identity': 'Identity',
    },
    'DecoupledFixed': {
        'AdjConv-impulse':  'Impulse',
        'AdjConv-mono':     'Monomial',
        'AdjConv-appr':     'PPR',
        'AdjConv-hk':       'HK',
        'AdjConv-gaussian': 'Gaussian',
    },
    'DecoupledVar': {
        'AdjConv':          'Var-Monomial',
        'HornerConv':       'Horner',
        'ChebConv':         'Chebyshev',
        'ChebConv2':        'ChebInterp',
        'ClenshawConv':     'Clenshaw',
        'BernConv':         'Bernstein',
    },
    'AdaGNN': {'AdaConv':   'AdaGNN'},
    'ACMGNN': {
        # 'ACMConv-1.0-low-high':     'FBGNNI',
        # 'ACMConv-2.0-low-high':     'FBGNNII',
        # 'ACMConv-1.0-low-high-id':  'ACMGNNI',
        # 'ACMConv-2.0-low-high-id':  'ACMGNNII',
        'ACMConv-2.0-low-high-id':  'ACMGNN',
    },
    'DecoupledFixedCompose': {
        'AdjiConv,AdjiConv-mono,mono': 'FAGNN',
        'Adji2Conv,Adji2Conv-gaussian,gaussian': 'G$^2$CN',
        'AdjDiffConv,AdjDiffConv-appr,appr': 'GNN-LF/HF',
    },
    'DecoupledVarCompose': {
        'AdjConv,ChebConv,BernConv': 'FiGURe',
    }
}
flst = []
for m in conv_repr.values():
    for c, name in m.items():
        flst.append(name)

df = t.groupby(['data', 'model', 'conv'])[m_cols].agg(['mean', 'std'])
df.columns = ['_'.join(col).strip() for col in df.columns.values]
df['delta'] = df['s_f1i_high_mean'] - df['s_f1i_low_mean']
df.reset_index(inplace=True)

df['name'] = df.apply(lambda row: conv_repr[row['model']][row['conv']] if (row['model'] in conv_repr) and (row['conv'] in conv_repr[row['model']]) else None, axis=1)
df.dropna(subset=['name'], inplace=True)
df.drop(columns=['model', 'conv'], inplace=True)
df = df[['data', 'name'] + df.columns.tolist()[1:-1]]
df

,data,name,num_high_mean,num_high_std,num_low_mean,num_low_std,f1micro_high_mean,f1micro_high_std,f1micro_low_mean,f1micro_low_std,f1micro_test_mean,f1micro_test_std,delta
0,actor,PPR,314.000000,17.000000,1206.000000,17.000000,34.897333,1.262113,34.904667,0.965054,34.912667,0.502556,-0.007333
1,actor,Gaussian,319.333333,17.009801,1200.666667,17.009801,37.694333,2.934441,35.867667,1.710697,36.271667,1.159164,1.826667
2,actor,HK,314.666667,17.009801,1205.333333,17.009801,37.363667,1.913040,35.151000,0.356219,35.614333,0.385587,2.212667
3,actor,Impulse,310.666667,12.342339,1209.333333,12.342339,25.071333,2.438311,23.541000,5.211034,23.838000,4.638309,1.530333
4,actor,Monomial,312.000000,15.099669,1208.000000,15.099669,35.660000,0.754650,35.999333,1.449973,35.943000,1.044948,-0.339333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,squirrel_filtered,ChebInterp,84.000000,6.000000,360.000000,6.000000,21.650333,5.699912,41.881000,2.655969,37.988000,1.064261,-20.230667
101,squirrel_filtered,Clenshaw,84.000000,5.567764,360.000000,5.567764,22.342667,18.273022,36.090333,2.110186,33.408333,5.007479,-13.747667
102,squirrel_filtered,Horner,84.000000,5.567764,360.000000,5.567764,25.500667,11.744395,36.183667,1.952592,34.234333,2.064232,-10.683000
103,squirrel_filtered,FiGURe,97.333333,8.020806,346.666667,8.020806,36.988333,8.074107,37.448667,3.316605,37.312333,4.043729,-0.460333


## Latex

In [19]:
dlst_acc = dlst
mlst = ['s_f1i_high_mean', 's_f1i_low_mean', 'delta']
acc_str = pd.DataFrame(index=flst, columns=pd.MultiIndex.from_product([dlst_acc, mlst]))

for row in acc_str.index:
    for data, met in acc_str.columns:
        mean = df.loc[(df['name'] == row) & (df['data'] == data), met].values
        if len(mean) > 0:
            if met == 'delta':
                acc_str.loc[row, (data, met)] = f"{mean[0]:+.2f}"
            else:
                acc_str.loc[row, (data, met)] = f"{mean[0]:.2f}"
        else:
            acc_str.loc[row, (data, met)] = "-"
acc_str

cora                                  citeseer  \
             f1micro_high_mean f1micro_low_mean  delta f1micro_high_mean   
Identity                 76.99            76.88  +0.10             69.68   
Impulse                  70.53            69.27  +1.26             79.51   
Monomial                 86.09            87.58  -1.49             80.56   
PPR                      85.87            88.74  -2.87             81.20   
HK                       76.20            74.45  +1.76             79.05   
Gaussian                 85.07            87.97  -2.90             79.26   
Var-Monomial             74.50            78.44  -3.94             82.54   
Horner                   85.37            87.90  -2.53             66.08   
Chebyshev                75.51            75.80  -0.29             71.17   
ChebInterp               87.35            86.01  +1.34             82.15   
Clenshaw                 83.07            86.71  -3.64             62.33   
Bernstein                43.47            52.42  -8.94             64.56   
AdaGNN                   85.35            87.73  -2.38             81.66   
ACMGNN                   58.36            59.71  -1.35             64.02   
FAGNN                    84.26            84.61  -0.35             79.47   
G$^2$CN                  79.01            80.35  -1.35             81.23   
GNN-LF/HF                77.59            77.17  +0.42             77.85   
FiGURe                   83.96            87.14  -3.19             78.59   

                                     chameleon_filtered                   \
             f1micro_low_mean  delta  f1micro_high_mean f1micro_low_mean   
Identity                71.20  -1.53              23.39            34.74   
Impulse                 74.24  +5.27              33.18            37.89   
Monomial                75.28  +5.28              34.24            35.91   
PPR                     75.93  +5.27              34.30            32.45   
HK                      75.57  +3.48              33.29            36.44   
Gaussian                75.04  +4.22              27.96            38.42   
Var-Monomial            75.33  +7.21              19.06            39.40   
Horner                  61.03  +5.05              19.00            38.45   
Chebyshev               71.69  -0.52              33.45            38.12   
ChebInterp              75.67  +6.48              34.73            46.06   
Clenshaw                55.16  +7.18              24.57            33.41   
Bernstein               66.90  -2.34              41.10            40.34   
AdaGNN                  75.89  +5.77              45.54            35.19   
ACMGNN                  62.68  +1.34              10.33            40.14   
FAGNN                   73.26  +6.21              25.16            40.51   
G$^2$CN                 75.80  +5.43              29.75            40.12   
GNN-LF/HF               73.39  +4.45              35.08            33.36   
FiGURe                  76.13  +2.47              28.84            43.72   

                     squirrel_filtered                           \
               delta f1micro_high_mean f1micro_low_mean   delta   
Identity      -11.35             18.65            33.83  -15.18   
Impulse        -4.71              9.24            41.42  -32.18   
Monomial       -1.67             30.92            32.76   -1.84   
PPR            +1.85              9.39            40.12  -30.73   
HK             -3.15              4.06            40.02  -35.96   
Gaussian      -10.46             32.20            41.43   -9.22   
Var-Monomial  -20.34             35.32            38.64   -3.33   
Horner        -19.45             25.50            36.18  -10.68   
Chebyshev      -4.67             29.43            35.85   -6.42   
ChebInterp    -11.33             21.65            41.88  -20.23   
Clenshaw       -8.85             22.34            36.09  -13.75   
Bernstein      +0.76             18.93            40.69  -21.76   
AdaGNN        +10.35              8.72        

In [20]:
dlst_eff = ["cora", "citeseer", "squirrel_filtered", "chameleon_filtered"]
for row in acc_str.index:
    values = acc_str.loc[row, dlst_eff].values.tolist()
    print("\t& " + row + " & " + " & ".join(values) + " \\\\")
print()

	& Identity & 76.99 & 76.88 & +0.10 & 69.68 & 71.20 & -1.53 & 18.65 & 33.83 & -15.18 & 23.39 & 34.74 & -11.35 \\
	& Impulse & 70.53 & 69.27 & +1.26 & 79.51 & 74.24 & +5.27 & 9.24 & 41.42 & -32.18 & 33.18 & 37.89 & -4.71 \\
	& Monomial & 86.09 & 87.58 & -1.49 & 80.56 & 75.28 & +5.28 & 30.92 & 32.76 & -1.84 & 34.24 & 35.91 & -1.67 \\
	& PPR & 85.87 & 88.74 & -2.87 & 81.20 & 75.93 & +5.27 & 9.39 & 40.12 & -30.73 & 34.30 & 32.45 & +1.85 \\
	& HK & 76.20 & 74.45 & +1.76 & 79.05 & 75.57 & +3.48 & 4.06 & 40.02 & -35.96 & 33.29 & 36.44 & -3.15 \\
	& Gaussian & 85.07 & 87.97 & -2.90 & 79.26 & 75.04 & +4.22 & 32.20 & 41.43 & -9.22 & 27.96 & 38.42 & -10.46 \\
	& Var-Monomial & 74.50 & 78.44 & -3.94 & 82.54 & 75.33 & +7.21 & 35.32 & 38.64 & -3.33 & 19.06 & 39.40 & -20.34 \\
	& Horner & 85.37 & 87.90 & -2.53 & 66.08 & 61.03 & +5.05 & 25.50 & 36.18 & -10.68 & 19.00 & 38.45 & -19.45 \\
	& Chebyshev & 75.51 & 75.80 & -0.29 & 71.17 & 71.69 & -0.52 & 29.43 & 35.85 & -6.42 & 33.45 & 38.12 & -4.67 \\
	& C